In [4]:
# Cell 1 (Proposed Pipeline) – Imports & configuration

import numpy as np
import pandas as pd

from xgboost import XGBRegressor
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error

from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
import time

# FFT for LF-ER feature
from numpy.fft import rfft

RANDOM_STATE = 42

WINDOW_SIZE = 30
STRIDE = 1

# Small epsilon for numerical stability (shared by fslope and LF-ER)
EPSILON = 1e-6

SENSOR_COLS = [f"s_{i}" for i in range(1, 22)]  # s_1 ... s_21
SETTING_COLS = ["os_1", "os_2", "os_3"]

ALL_VAR_COLS = SENSOR_COLS + SETTING_COLS

print("Proposed pipeline configuration loaded.")
print(f"RANDOM_STATE = {RANDOM_STATE}")
print(f"WINDOW_SIZE  = {WINDOW_SIZE}")
print(f"STRIDE       = {STRIDE}")
print(f"# variables for fslope / LF-ER: {len(ALL_VAR_COLS)}")


Proposed pipeline configuration loaded.
RANDOM_STATE = 42
WINDOW_SIZE  = 30
STRIDE       = 1
# variables for fslope / LF-ER: 24


In [5]:
# Cell 2 – Load FD001 and prepare RUL labels

# -------------------------------------------------------------------
# File paths (adjust these if your folder structure is different)
# -------------------------------------------------------------------
TRAIN_PATH = "C:/Users/hrast/OneDrive/Desktop/Minor Project/CMaps/train_FD001.txt"
TEST_PATH  = "C:/Users/hrast/OneDrive/Desktop/Minor Project/CMaps/test_FD001.txt"
RUL_PATH   = "C:/Users/hrast/OneDrive/Desktop/Minor Project/CMaps/RUL_FD001.txt"


def load_fd001(train_path=TRAIN_PATH, test_path=TEST_PATH, rul_path=RUL_PATH):
    """
    Load the NASA C-MAPSS FD001 dataset:
      - training trajectories,
      - test trajectories,
      - final RUL values for each test engine.
    """
    col_names = (
        ["unit_nr", "time_cycles"] +
        [f"os_{i}" for i in range(1, 4)] +      # os_1, os_2, os_3
        [f"s_{i}" for i in range(1, 22)]        # s_1 ... s_21
    )

    # sep=r"\s+" handles variable whitespace between columns
    train_raw = pd.read_csv(train_path, sep=r"\s+", header=None)
    test_raw  = pd.read_csv(test_path,  sep=r"\s+", header=None)

    # In case there are extra trailing empty columns, trim to expected length
    if train_raw.shape[1] > len(col_names):
        train_raw = train_raw.iloc[:, :len(col_names)]
    if test_raw.shape[1] > len(col_names):
        test_raw = test_raw.iloc[:, :len(col_names)]

    train_raw.columns = col_names
    test_raw.columns  = col_names

    # RUL for test engines (one row per engine)
    rul_test_final = pd.read_csv(rul_path, sep=r"\s+", header=None)
    rul_test_final.columns = ["RUL"]

    return train_raw, test_raw, rul_test_final


def add_rul_to_train(train_df):
    """
    For each unit_nr in the training set, compute remaining useful life (RUL):

        RUL(t) = max_cycle(unit) - time_cycles(t)

    and append it as a new column 'RUL'.
    """
    train_df = train_df.copy()
    max_cycles = train_df.groupby("unit_nr")["time_cycles"].transform("max")
    train_df["RUL"] = max_cycles - train_df["time_cycles"]
    return train_df


def get_test_final_cycles(test_df, rul_df):
    """
    For the test set:
      - find the last observed cycle for each engine,
      - keep only that row,
      - attach the corresponding RUL from rul_df.

    Returns a DataFrame with exactly one row per test engine.
    """
    # Index of last cycle for each unit
    idx_last = test_df.groupby("unit_nr")["time_cycles"].idxmax()
    test_last = test_df.loc[idx_last].copy()

    # Sort by unit_nr to align with RUL file ordering
    test_last = test_last.sort_values("unit_nr").reset_index(drop=True)
    test_last["RUL"] = rul_df["RUL"].values

    return test_last


# -------------------------------------------------------------------
# Actually load data and build labeled train/test summaries
# -------------------------------------------------------------------
start_time = time.time()

train_raw, test_raw, rul_test_final = load_fd001()
train_labeled = add_rul_to_train(train_raw)
test_last_labeled = get_test_final_cycles(test_raw, rul_test_final)

end_time = time.time()

print("FD001 data loaded and labeled.")
print(f"train_raw shape       : {train_raw.shape}")
print(f"test_raw shape        : {test_raw.shape}")
print(f"rul_test_final shape  : {rul_test_final.shape}")
print(f"train_labeled shape   : {train_labeled.shape}")
print(f"test_last_labeled shape (one row per engine): {test_last_labeled.shape}")
print(f"Data loading & labeling time: {end_time - start_time:.3f} seconds")


FD001 data loaded and labeled.
train_raw shape       : (20631, 26)
test_raw shape        : (13096, 26)
rul_test_final shape  : (100, 1)
train_labeled shape   : (20631, 27)
test_last_labeled shape (one row per engine): (100, 27)
Data loading & labeling time: 0.398 seconds


In [6]:
# Cell 3 – fslope (normalized windowed trend slope) feature extraction

def compute_fslope(x_window, t_idx, t_mean, denom_t, epsilon=EPSILON):
    """
    Compute the normalized windowed trend slope (fslope) for a 1D window.

    We fit a simple linear regression:
        x ≈ alpha + beta * t
    with equally spaced t indices (0, 1, ..., w-1).

    Slope beta is normalized by the mean magnitude of x in the window:
        fslope = beta / (|mean(x)| + epsilon)

    Parameters
    ----------
    x_window : array-like, shape (w,)
        Values of a single sensor/setting in the current window.
    t_idx : np.ndarray, shape (w,)
        Precomputed time indices for the window (0..w-1).
    t_mean : float
        Mean of t_idx.
    denom_t : float
        Sum of squared deviations of t_idx: sum((t - t_mean)^2).
    epsilon : float
        Small constant to avoid division by zero.

    Returns
    -------
    fslope : float
        Normalized windowed slope for this window.
    """
    x = np.asarray(x_window, dtype=np.float64)
    x_mean = x.mean()

    # Numerator of slope beta: sum( (t - t_mean) * (x - x_mean) )
    num = np.dot(t_idx - t_mean, x - x_mean)
    beta = num / (denom_t + 1e-12)

    # Normalized slope
    fslope = beta / (np.abs(x_mean) + epsilon)
    return fslope


def generate_rolling_features_for_unit_fslope(
    df_unit,
    window=WINDOW_SIZE,
    stride=STRIDE,
    epsilon=EPSILON,
):
    """
    Per-unit rolling feature generator with baseline stats + fslope.

    For a single engine (unit_nr), this function:
      - sorts by time_cycles,
      - uses a sliding window of length `window` and step `stride`,
      - for EACH variable in SENSOR_COLS + SETTING_COLS:
          * computes baseline statistics:
                mean, std, min, max, skew, kurtosis
          * additionally computes fslope (normalized trend slope),
      - labels each window with the RUL at the last cycle in the window,
      - records unit_nr and time_cycles of the last cycle.

    Returns
    -------
    features_df : pd.DataFrame
        One row per window, with columns:
          - 'unit_nr', 'time_cycles'
          - baseline stats for each variable
          - '{var}_fslope' for each variable
          - 'RUL' (label at last cycle in window)
    """
    df_unit = df_unit.sort_values("time_cycles").reset_index(drop=True)
    n_cycles = len(df_unit)

    if n_cycles < window:
        # Not enough cycles to form a single window
        return pd.DataFrame()

    # Precompute time indices for the regression (0, 1, ..., window-1)
    t_idx = np.arange(window, dtype=np.float64)
    t_mean = t_idx.mean()
    denom_t = np.sum((t_idx - t_mean) ** 2)

    rows = []
    var_cols = SENSOR_COLS + SETTING_COLS

    for start in range(0, n_cycles - window + 1, stride):
        end = start + window
        window_df = df_unit.iloc[start:end]

        last_row = window_df.iloc[-1]
        row_dict = {
            "unit_nr": last_row["unit_nr"],
            "time_cycles": last_row["time_cycles"],
        }

        # Compute features for each variable in the window
        for col in var_cols:
            x = window_df[col].values.astype(np.float64)

            # Baseline rolling stats
            x_mean = x.mean()
            x_std = x.std(ddof=0)
            x_min = x.min()
            x_max = x.max()
            x_skew = skew(x)
            x_kurt = kurtosis(x)

            row_dict[f"{col}_mean"] = x_mean
            row_dict[f"{col}_std"] = x_std
            row_dict[f"{col}_min"] = x_min
            row_dict[f"{col}_max"] = x_max
            row_dict[f"{col}_skew"] = x_skew
            row_dict[f"{col}_kurt"] = x_kurt

            # New fslope feature
            row_dict[f"{col}_fslope"] = compute_fslope(
                x_window=x,
                t_idx=t_idx,
                t_mean=t_mean,
                denom_t=denom_t,
                epsilon=epsilon,
            )

        # RUL label taken at the last cycle in the window
        row_dict["RUL"] = last_row["RUL"]

        rows.append(row_dict)

    features_df = pd.DataFrame(rows)
    return features_df


def generate_rolling_features_fslope(
    df,
    window=WINDOW_SIZE,
    stride=STRIDE,
    epsilon=EPSILON,
):
    """
    Wrapper over generate_rolling_features_for_unit_fslope for all engines.

    Parameters
    ----------
    df : pd.DataFrame
        Full labeled dataset (e.g., train_labeled or test_with_dummy_rul),
        containing 'unit_nr', 'time_cycles', 'RUL', SENSOR_COLS, SETTING_COLS.
    window : int
        Sliding window length (same as baseline).
    stride : int
        Sliding window step (same as baseline).
    epsilon : float
        Small constant for fslope normalization.

    Returns
    -------
    features_all : pd.DataFrame
        Concatenation of per-unit features, sorted by unit_nr and time_cycles.
    """
    all_features = []

    for unit_id, df_unit in df.groupby("unit_nr"):
        feats_unit = generate_rolling_features_for_unit_fslope(
            df_unit,
            window=window,
            stride=stride,
            epsilon=epsilon,
        )
        if not feats_unit.empty:
            all_features.append(feats_unit)

    if not all_features:
        return pd.DataFrame()

    features_all = pd.concat(all_features, axis=0, ignore_index=True)
    features_all = features_all.sort_values(["unit_nr", "time_cycles"]).reset_index(drop=True)
    return features_all


print("fslope helpers and rolling feature generators are defined.")

fslope helpers and rolling feature generators are defined.


In [7]:
# Cell 4 – Train feature extraction with fslope (proposed pipeline)

start_time = time.time()

# Generate sliding-window features with baseline stats + fslope
train_features_fslope = generate_rolling_features_fslope(
    train_labeled,
    window=WINDOW_SIZE,
    stride=STRIDE,
    epsilon=EPSILON,
)

end_time = time.time()
fslope_train_time = end_time - start_time

print("Proposed (fslope) TRAIN feature extraction completed.")
print(f"Proposed (fslope) TRAIN feature extraction time: {fslope_train_time:.3f} seconds")
print(f"train_features_fslope shape: {train_features_fslope.shape}")

# Inspect feature columns (exclude ID/label columns)
non_feature_cols_fslope = ["unit_nr", "time_cycles", "RUL"]
feature_cols_fslope = [c for c in train_features_fslope.columns if c not in non_feature_cols_fslope]

print(f"Number of features (proposed fslope): {len(feature_cols_fslope)}")
print("Example feature columns (first 15):")
print(feature_cols_fslope[:15])

C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\4161080762.py:106: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_skew = skew(x)
C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\4161080762.py:107: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_kurt = kurtosis(x)


Proposed (fslope) TRAIN feature extraction completed.
Proposed (fslope) TRAIN feature extraction time: 665.316 seconds
train_features_fslope shape: (17731, 171)
Number of features (proposed fslope): 168
Example feature columns (first 15):
['s_1_mean', 's_1_std', 's_1_min', 's_1_max', 's_1_skew', 's_1_kurt', 's_1_fslope', 's_2_mean', 's_2_std', 's_2_min', 's_2_max', 's_2_skew', 's_2_kurt', 's_2_fslope', 's_3_mean']


In [8]:
# Cell 5 – Build X, y, groups for the fslope-augmented proposed pipeline

# If for some reason feature_cols_fslope is not defined (e.g., you re-run cells out of order),
# we reconstruct it here to be robust.
non_feature_cols_fslope = ["unit_nr", "time_cycles", "RUL"]
if "feature_cols_fslope" not in globals():
    feature_cols_fslope = [
        c for c in train_features_fslope.columns
        if c not in non_feature_cols_fslope
    ]

# Design matrix (features), target (RUL), and groups (unit_nr)
X_fslope = train_features_fslope[feature_cols_fslope].values
y_fslope = train_features_fslope["RUL"].values
groups_fslope = train_features_fslope["unit_nr"].values

print("Proposed (fslope) design matrix constructed.")
print(f"X_fslope shape      : {X_fslope.shape}")
print(f"y_fslope shape      : {y_fslope.shape}")
print(f"groups_fslope shape : {groups_fslope.shape}")
print(f"Number of features (fslope): {len(feature_cols_fslope)}")
print("Example feature columns (first 15):")
print(feature_cols_fslope[:15])

Proposed (fslope) design matrix constructed.
X_fslope shape      : (17731, 168)
y_fslope shape      : (17731,)
groups_fslope shape : (17731,)
Number of features (fslope): 168
Example feature columns (first 15):
['s_1_mean', 's_1_std', 's_1_min', 's_1_max', 's_1_skew', 's_1_kurt', 's_1_fslope', 's_2_mean', 's_2_std', 's_2_min', 's_2_max', 's_2_skew', 's_2_kurt', 's_2_fslope', 's_3_mean']


In [9]:
# Cell 6 – GroupKFold CV with XGBoost (proposed fslope features)

def rmse(y_true, y_pred):
    """Root Mean Squared Error."""
    return np.sqrt(mean_squared_error(y_true, y_pred))


gkf = GroupKFold(n_splits=5)

rmse_scores_fslope = []
mae_scores_fslope = []

cv_start_time = time.time()

for fold, (train_idx, val_idx) in enumerate(
    gkf.split(X_fslope, y_fslope, groups_fslope), start=1
):
    X_tr, X_val = X_fslope[train_idx], X_fslope[val_idx]
    y_tr, y_val = y_fslope[train_idx], y_fslope[val_idx]

    model_fslope_cv = XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=RANDOM_STATE,
    )

    model_fslope_cv.fit(X_tr, y_tr)
    y_val_pred = model_fslope_cv.predict(X_val)

    fold_rmse = rmse(y_val, y_val_pred)
    fold_mae = mean_absolute_error(y_val, y_val_pred)

    rmse_scores_fslope.append(fold_rmse)
    mae_scores_fslope.append(fold_mae)

    print(f"Fold {fold}: RMSE = {fold_rmse:.3f}, MAE = {fold_mae:.3f}")

cv_end_time = time.time()
cv_time_fslope = cv_end_time - cv_start_time

rmse_mean_fslope = np.mean(rmse_scores_fslope)
rmse_std_fslope = np.std(rmse_scores_fslope)
mae_mean_fslope = np.mean(mae_scores_fslope)
mae_std_fslope = np.std(mae_scores_fslope)

print("\nProposed (fslope) CV results:")
print(f"  RMSE mean = {rmse_mean_fslope:.3f}, std = {rmse_std_fslope:.3f}")
print(f"  MAE  mean = {mae_mean_fslope:.3f}, std = {mae_std_fslope:.3f}")
print(f"Proposed (fslope) CV time (5 folds): {cv_time_fslope:.3f} seconds")

Fold 1: RMSE = 38.298, MAE = 23.874
Fold 2: RMSE = 32.603, MAE = 20.757
Fold 3: RMSE = 31.603, MAE = 20.011
Fold 4: RMSE = 33.677, MAE = 22.375
Fold 5: RMSE = 30.326, MAE = 20.997

Proposed (fslope) CV results:
  RMSE mean = 33.301, std = 2.732
  MAE  mean = 21.603, std = 1.369
Proposed (fslope) CV time (5 folds): 17.535 seconds


In [10]:
# Cell 7 – Final XGBoost model trained on full fslope feature set

model_fslope = XGBRegressor(
    n_estimators=400,           # slightly larger than CV (300), like your baseline final model
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

start_time = time.time()

# Train on ALL fslope-augmented training windows
model_fslope.fit(X_fslope, y_fslope)

end_time = time.time()
fslope_train_model_time = end_time - start_time

print("Final proposed (fslope) model trained successfully.")
print(f"Training time (full fslope features): {fslope_train_model_time:.3f} seconds")
print(f"Number of training samples: {X_fslope.shape[0]}")
print(f"Number of features        : {X_fslope.shape[1]}")

Final proposed (fslope) model trained successfully.
Training time (full fslope features): 5.511 seconds
Number of training samples: 17731
Number of features        : 168


In [11]:
# Cell 8 – TEST feature extraction with fslope (proposed pipeline)

def add_dummy_rul(test_df):
    """
    Add a dummy RUL column to the test set so that the same
    rolling feature generator can be reused.

    The dummy RUL will later be replaced with the true RUL values
    from rul_test_final for the last cycle of each engine.
    """
    df = test_df.copy()
    df["RUL"] = 0  # placeholder, not used directly for test labeling
    return df


# Add dummy RUL to test trajectories
test_with_dummy_rul = add_dummy_rul(test_raw)

# Time the TEST feature extraction with fslope
start_time = time.time()

test_features_fslope_full = generate_rolling_features_fslope(
    test_with_dummy_rul,
    window=WINDOW_SIZE,
    stride=STRIDE,
    epsilon=EPSILON,
)

end_time = time.time()
fslope_test_time = end_time - start_time

print("Proposed (fslope) TEST feature extraction completed.")
print(f"Proposed (fslope) TEST feature extraction time: {fslope_test_time:.3f} seconds")
print(f"test_features_fslope_full shape: {test_features_fslope_full.shape}")

C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\4161080762.py:106: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_skew = skew(x)
C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\4161080762.py:107: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_kurt = kurtosis(x)


Proposed (fslope) TEST feature extraction completed.
Proposed (fslope) TEST feature extraction time: 383.331 seconds
test_features_fslope_full shape: (10196, 171)


In [12]:
# Cell 9 – TEST evaluation for the proposed fslope model

# 1) For each test engine, keep ONLY the last available window
idx_last_per_unit_fslope = (
    test_features_fslope_full.groupby("unit_nr")["time_cycles"].idxmax()
)

test_features_fslope_last = (
    test_features_fslope_full
        .loc[idx_last_per_unit_fslope]
        .sort_values("unit_nr")
        .reset_index(drop=True)
)

# 2) Replace dummy RUL with true final RUL from rul_test_final
test_features_fslope_last["RUL"] = rul_test_final["RUL"].reset_index(drop=True)

# 3) Build X_test and y_test using the SAME feature columns as training
#    (feature_cols_fslope was defined in Cell 4/5)
X_test_fslope = test_features_fslope_last[feature_cols_fslope].values
y_test_fslope = test_features_fslope_last["RUL"].values

# 4) Predict with the final fslope model and compute metrics
y_test_fslope_pred = model_fslope.predict(X_test_fslope)

test_rmse_fslope = rmse(y_test_fslope, y_test_fslope_pred)
test_mae_fslope  = mean_absolute_error(y_test_fslope, y_test_fslope_pred)

print("Proposed (fslope) TEST evaluation:")
print(f"  test_features_fslope_last shape: {test_features_fslope_last.shape}")
print(f"  X_test_fslope shape            : {X_test_fslope.shape}")
print(f"  y_test_fslope shape            : {y_test_fslope.shape}")
print(f"  TEST RMSE (fslope)             : {test_rmse_fslope:.3f}")
print(f"  TEST MAE  (fslope)             : {test_mae_fslope:.3f}")

Proposed (fslope) TEST evaluation:
  test_features_fslope_last shape: (100, 171)
  X_test_fslope shape            : (100, 168)
  y_test_fslope shape            : (100,)
  TEST RMSE (fslope)             : 22.350
  TEST MAE  (fslope)             : 17.190


In [13]:
# Cell 10 – LF-ER (low-frequency energy ratio) feature extraction

def compute_lfer(x_window, low_freq_ratio=0.25, epsilon=EPSILON):
    """
    Compute the low-frequency energy ratio (LF-ER) for a 1D window.

    Steps:
      1) Compute FFT of the windowed signal using rfft (real FFT).
      2) Compute spectral energy |X(f)|^2 for each frequency bin.
      3) Define a low-frequency band as the first K non-DC bins:
            K = max(1, floor(low_freq_ratio * (N_bins - 1)))
         where N_bins = len(power_spectrum).
      4) LF-ER = (sum of low-frequency energy) / (total energy + epsilon).

    Parameters
    ----------
    x_window : array-like, shape (w,)
        Values of a single sensor/setting in the current window.
    low_freq_ratio : float
        Fraction of non-DC FFT bins to treat as "low frequency" (0 < ratio <= 1).
    epsilon : float
        Small constant to avoid division by zero.

    Returns
    -------
    lfer : float
        Low-frequency energy ratio for this window.
    """
    x = np.asarray(x_window, dtype=np.float64)

    # Optionally, we could remove the mean to focus on oscillatory behavior.
    # Here we follow a simple, consistent approach:
    X = rfft(x)
    power = np.abs(X) ** 2

    total_energy = power.sum()

    if len(power) <= 1:
        # Degenerate FFT (e.g., constant or extremely short signal)
        return 0.0

    # Exclude DC (index 0) when defining low-frequency band
    n_non_dc = len(power) - 1
    k_low = int(np.floor(low_freq_ratio * n_non_dc))
    k_low = max(1, k_low)  # at least 1 low-frequency bin

    # Low-frequency bins: indices 1 .. k_low (inclusive)
    low_energy = power[1 : 1 + k_low].sum()

    lfer = low_energy / (total_energy + epsilon)
    return float(lfer)


def generate_rolling_features_for_unit_lfer(
    df_unit,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,
    epsilon=EPSILON,
):
    """
    Per-unit rolling feature generator with baseline stats + LF-ER.

    For a single engine (unit_nr), this function:
      - sorts by time_cycles,
      - uses a sliding window of length `window` and step `stride`,
      - for EACH variable in SENSOR_COLS + SETTING_COLS:
          * computes baseline statistics:
                mean, std, min, max, skew, kurtosis
          * additionally computes LF-ER (low-frequency energy ratio),
      - labels each window with the RUL at the last cycle in the window,
      - records unit_nr and time_cycles of the last cycle.

    Returns
    -------
    features_df : pd.DataFrame
        One row per window, with columns:
          - 'unit_nr', 'time_cycles'
          - baseline stats for each variable
          - '{var}_lfer' for each variable
          - 'RUL' (label at last cycle in window)
    """
    df_unit = df_unit.sort_values("time_cycles").reset_index(drop=True)
    n_cycles = len(df_unit)

    if n_cycles < window:
        # Not enough cycles to form a single window
        return pd.DataFrame()

    rows = []
    var_cols = SENSOR_COLS + SETTING_COLS

    for start in range(0, n_cycles - window + 1, stride):
        end = start + window
        window_df = df_unit.iloc[start:end]

        last_row = window_df.iloc[-1]
        row_dict = {
            "unit_nr": last_row["unit_nr"],
            "time_cycles": last_row["time_cycles"],
        }

        # Compute features for each variable in the window
        for col in var_cols:
            x = window_df[col].values.astype(np.float64)

            # Baseline rolling stats
            x_mean = x.mean()
            x_std = x.std(ddof=0)
            x_min = x.min()
            x_max = x.max()
            x_skew = skew(x)
            x_kurt = kurtosis(x)

            row_dict[f"{col}_mean"] = x_mean
            row_dict[f"{col}_std"] = x_std
            row_dict[f"{col}_min"] = x_min
            row_dict[f"{col}_max"] = x_max
            row_dict[f"{col}_skew"] = x_skew
            row_dict[f"{col}_kurt"] = x_kurt

            # New LF-ER feature
            row_dict[f"{col}_lfer"] = compute_lfer(
                x_window=x,
                low_freq_ratio=low_freq_ratio,
                epsilon=epsilon,
            )

        # RUL label taken at the last cycle in the window
        row_dict["RUL"] = last_row["RUL"]

        rows.append(row_dict)

    features_df = pd.DataFrame(rows)
    return features_df


def generate_rolling_features_lfer(
    df,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,
    epsilon=EPSILON,
):
    """
    Wrapper over generate_rolling_features_for_unit_lfer for all engines.

    Parameters
    ----------
    df : pd.DataFrame
        Full labeled dataset (e.g., train_labeled or test_with_dummy_rul),
        containing 'unit_nr', 'time_cycles', 'RUL', SENSOR_COLS, SETTING_COLS.
    window : int
        Sliding window length (same as baseline).
    stride : int
        Sliding window step (same as baseline).
    low_freq_ratio : float
        Fraction of FFT non-DC bins treated as low frequency.
    epsilon : float
        Small constant for LF-ER normalization.

    Returns
    -------
    features_all : pd.DataFrame
        Concatenation of per-unit features, sorted by unit_nr and time_cycles.
    """
    all_features = []

    for unit_id, df_unit in df.groupby("unit_nr"):
        feats_unit = generate_rolling_features_for_unit_lfer(
            df_unit,
            window=window,
            stride=stride,
            low_freq_ratio=low_freq_ratio,
            epsilon=epsilon,
        )
        if not feats_unit.empty:
            all_features.append(feats_unit)

    if not all_features:
        return pd.DataFrame()

    features_all = pd.concat(all_features, axis=0, ignore_index=True)
    features_all = features_all.sort_values(["unit_nr", "time_cycles"]).reset_index(drop=True)
    return features_all


print("LF-ER helpers and rolling feature generators (LFER-only) are defined.")

LF-ER helpers and rolling feature generators (LFER-only) are defined.


In [14]:
# Cell 11 – Train feature extraction with LF-ER (proposed LF-ER-only pipeline)

# Generate sliding-window features with baseline stats + LF-ER
start_time = time.time()

train_features_lfer = generate_rolling_features_lfer(
    train_labeled,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,  # first 25% of non-DC FFT bins as "low frequency"
    epsilon=EPSILON,
)

end_time = time.time()
lfer_train_time = end_time - start_time

print("Proposed (LF-ER) TRAIN feature extraction completed.")
print(f"Proposed (LF-ER) TRAIN feature extraction time: {lfer_train_time:.3f} seconds")
print(f"train_features_lfer shape: {train_features_lfer.shape}")

# Inspect feature columns (exclude ID/label columns)
non_feature_cols_lfer = ["unit_nr", "time_cycles", "RUL"]
feature_cols_lfer = [c for c in train_features_lfer.columns if c not in non_feature_cols_lfer]

print(f"Number of features (proposed LF-ER): {len(feature_cols_lfer)}")
print("Example feature columns (first 15):")
print(feature_cols_lfer[:15])

C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\120442753.py:112: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_skew = skew(x)
C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\120442753.py:113: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_kurt = kurtosis(x)


Proposed (LF-ER) TRAIN feature extraction completed.
Proposed (LF-ER) TRAIN feature extraction time: 691.601 seconds
train_features_lfer shape: (17731, 171)
Number of features (proposed LF-ER): 168
Example feature columns (first 15):
['s_1_mean', 's_1_std', 's_1_min', 's_1_max', 's_1_skew', 's_1_kurt', 's_1_lfer', 's_2_mean', 's_2_std', 's_2_min', 's_2_max', 's_2_skew', 's_2_kurt', 's_2_lfer', 's_3_mean']


In [15]:
# Cell 12 – Build X, y, groups for the LF-ER-augmented pipeline

# Non-feature columns: identifiers and label
non_feature_cols_lfer = ["unit_nr", "time_cycles", "RUL"]

# Rebuild feature_cols_lfer defensively (in case of re-runs)
feature_cols_lfer = [
    c for c in train_features_lfer.columns
    if c not in non_feature_cols_lfer
]

# Design matrix (features), target (RUL), and groups (unit_nr)
X_lfer = train_features_lfer[feature_cols_lfer].values
y_lfer = train_features_lfer["RUL"].values
groups_lfer = train_features_lfer["unit_nr"].values

print("Proposed (LF-ER) design matrix constructed.")
print(f"X_lfer shape      : {X_lfer.shape}")
print(f"y_lfer shape      : {y_lfer.shape}")
print(f"groups_lfer shape : {groups_lfer.shape}")
print(f"Number of features (LF-ER): {len(feature_cols_lfer)}")
print("Example feature columns (first 15):")
print(feature_cols_lfer[:15])

Proposed (LF-ER) design matrix constructed.
X_lfer shape      : (17731, 168)
y_lfer shape      : (17731,)
groups_lfer shape : (17731,)
Number of features (LF-ER): 168
Example feature columns (first 15):
['s_1_mean', 's_1_std', 's_1_min', 's_1_max', 's_1_skew', 's_1_kurt', 's_1_lfer', 's_2_mean', 's_2_std', 's_2_min', 's_2_max', 's_2_skew', 's_2_kurt', 's_2_lfer', 's_3_mean']


In [16]:
# Cell 13 – GroupKFold CV with XGBoost (proposed LF-ER features)

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))


gkf_lfer = GroupKFold(n_splits=5)

rmse_scores_lfer = []
mae_scores_lfer = []

cv_start_time_lfer = time.time()

for fold, (train_idx, val_idx) in enumerate(
    gkf_lfer.split(X_lfer, y_lfer, groups_lfer), start=1
):
    X_tr, X_val = X_lfer[train_idx], X_lfer[val_idx]
    y_tr, y_val = y_lfer[train_idx], y_lfer[val_idx]

    model_lfer_cv = XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=RANDOM_STATE,
    )

    model_lfer_cv.fit(X_tr, y_tr)
    y_val_pred = model_lfer_cv.predict(X_val)

    fold_rmse = rmse(y_val, y_val_pred)
    fold_mae = mean_absolute_error(y_val, y_val_pred)

    rmse_scores_lfer.append(fold_rmse)
    mae_scores_lfer.append(fold_mae)

    print(f"Fold {fold}: RMSE = {fold_rmse:.3f}, MAE = {fold_mae:.3f}")

cv_end_time_lfer = time.time()
cv_time_lfer = cv_end_time_lfer - cv_start_time_lfer

rmse_mean_lfer = np.mean(rmse_scores_lfer)
rmse_std_lfer = np.std(rmse_scores_lfer)
mae_mean_lfer = np.mean(mae_scores_lfer)
mae_std_lfer = np.std(mae_scores_lfer)

print("\nProposed (LF-ER) CV results:")
print(f"  RMSE mean = {rmse_mean_lfer:.3f}, std = {rmse_std_lfer:.3f}")
print(f"  MAE  mean = {mae_mean_lfer:.3f}, std = {mae_std_lfer:.3f}")
print(f"Proposed (LF-ER) CV time (5 folds): {cv_time_lfer:.3f} seconds")


Fold 1: RMSE = 43.135, MAE = 27.614
Fold 2: RMSE = 37.270, MAE = 24.917
Fold 3: RMSE = 35.514, MAE = 23.420
Fold 4: RMSE = 38.462, MAE = 26.377
Fold 5: RMSE = 33.803, MAE = 23.605

Proposed (LF-ER) CV results:
  RMSE mean = 37.637, std = 3.170
  MAE  mean = 25.187, std = 1.613
Proposed (LF-ER) CV time (5 folds): 16.657 seconds


In [17]:
# Cell 14 – Final XGBoost model trained on full LF-ER feature set

model_lfer = XGBRegressor(
    n_estimators=400,           # match final models: a bit larger than CV
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

start_time = time.time()

# Train on ALL LF-ER-augmented training windows
model_lfer.fit(X_lfer, y_lfer)

end_time = time.time()
lfer_train_model_time = end_time - start_time

print("Final proposed (LF-ER) model trained successfully.")
print(f"Training time (full LF-ER features): {lfer_train_model_time:.3f} seconds")
print(f"Number of training samples: {X_lfer.shape[0]}")
print(f"Number of features        : {X_lfer.shape[1]}")

Final proposed (LF-ER) model trained successfully.
Training time (full LF-ER features): 5.074 seconds
Number of training samples: 17731
Number of features        : 168


In [18]:
# Cell 15 – TEST feature extraction with LF-ER (proposed LF-ER-only pipeline)

# Ensure we have a test DataFrame with a dummy RUL column (reused from fslope part)
def add_dummy_rul(test_df):
    """
    Add a dummy RUL column to the test set so that the same
    rolling feature generator can be reused.

    The dummy RUL will later be replaced with the true RUL values
    from rul_test_final for the last cycle of each engine.
    """
    df = test_df.copy()
    df["RUL"] = 0  # placeholder, not used directly for test labeling
    return df


# If test_with_dummy_rul was already created for fslope, reuse it; otherwise create it.
if "test_with_dummy_rul" not in globals():
    test_with_dummy_rul = add_dummy_rul(test_raw)

# Time the TEST feature extraction with LF-ER
start_time = time.time()

test_features_lfer_full = generate_rolling_features_lfer(
    test_with_dummy_rul,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,
    epsilon=EPSILON,
)

end_time = time.time()
lfer_test_time = end_time - start_time

print("Proposed (LF-ER) TEST feature extraction completed.")
print(f"Proposed (LF-ER) TEST feature extraction time: {lfer_test_time:.3f} seconds")
print(f"test_features_lfer_full shape: {test_features_lfer_full.shape}")

C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\120442753.py:112: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_skew = skew(x)
C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\120442753.py:113: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_kurt = kurtosis(x)


Proposed (LF-ER) TEST feature extraction completed.
Proposed (LF-ER) TEST feature extraction time: 395.505 seconds
test_features_lfer_full shape: (10196, 171)


In [19]:
# Cell 16 – TEST evaluation for the proposed LF-ER model

# 1) For each test engine, keep ONLY the last available window
idx_last_per_unit_lfer = (
    test_features_lfer_full.groupby("unit_nr")["time_cycles"].idxmax()
)

test_features_lfer_last = (
    test_features_lfer_full
        .loc[idx_last_per_unit_lfer]
        .sort_values("unit_nr")
        .reset_index(drop=True)
)

# 2) Replace dummy RUL with true final RUL from rul_test_final
test_features_lfer_last["RUL"] = rul_test_final["RUL"].reset_index(drop=True)

# 3) Build X_test and y_test using the SAME feature columns as training
X_test_lfer = test_features_lfer_last[feature_cols_lfer].values
y_test_lfer = test_features_lfer_last["RUL"].values

# 4) Predict with the final LF-ER model and compute metrics
y_test_lfer_pred = model_lfer.predict(X_test_lfer)

test_rmse_lfer = rmse(y_test_lfer, y_test_lfer_pred)
test_mae_lfer  = mean_absolute_error(y_test_lfer, y_test_lfer_pred)

print("Proposed (LF-ER) TEST evaluation:")
print(f"  test_features_lfer_last shape: {test_features_lfer_last.shape}")
print(f"  X_test_lfer shape            : {X_test_lfer.shape}")
print(f"  y_test_lfer shape            : {y_test_lfer.shape}")
print(f"  TEST RMSE (LF-ER)            : {test_rmse_lfer:.3f}")
print(f"  TEST MAE  (LF-ER)            : {test_mae_lfer:.3f}")

Proposed (LF-ER) TEST evaluation:
  test_features_lfer_last shape: (100, 171)
  X_test_lfer shape            : (100, 168)
  y_test_lfer shape            : (100,)
  TEST RMSE (LF-ER)            : 27.920
  TEST MAE  (LF-ER)            : 20.035


In [20]:
# Cell 17 – Combined rolling features: baseline stats + fslope + LF-ER

def generate_rolling_features_for_unit_fslope_lfer(
    df_unit,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,
    epsilon=EPSILON,
):
    """
    Per-unit rolling feature generator with:
      - baseline statistics,
      - fslope (normalized trend slope),
      - LF-ER (low-frequency energy ratio).

    For a single engine (unit_nr), this function:
      - sorts by time_cycles,
      - uses a sliding window of length `window` and step `stride`,
      - for EACH variable in SENSOR_COLS + SETTING_COLS:
          * baseline stats:
                mean, std, min, max, skew, kurtosis
          * fslope:
                normalized linear trend slope over the window
          * lfer:
                low-frequency energy ratio from FFT
      - labels each window with the RUL at the last cycle in the window,
      - records unit_nr and time_cycles of the last cycle.

    Returns
    -------
    features_df : pd.DataFrame
        One row per window, with columns:
          - 'unit_nr', 'time_cycles'
          - '{var}_mean', '{var}_std', '{var}_min', '{var}_max',
            '{var}_skew', '{var}_kurt'
          - '{var}_fslope'
          - '{var}_lfer'
          - 'RUL'
    """
    df_unit = df_unit.sort_values("time_cycles").reset_index(drop=True)
    n_cycles = len(df_unit)

    if n_cycles < window:
        # Not enough cycles to form a single window
        return pd.DataFrame()

    rows = []
    var_cols = SENSOR_COLS + SETTING_COLS

    # Precompute time indices for fslope regression (0, 1, ..., window-1)
    t_idx = np.arange(window, dtype=np.float64)
    t_mean = t_idx.mean()
    denom_t = np.sum((t_idx - t_mean) ** 2)

    for start in range(0, n_cycles - window + 1, stride):
        end = start + window
        window_df = df_unit.iloc[start:end]

        last_row = window_df.iloc[-1]
        row_dict = {
            "unit_nr": last_row["unit_nr"],
            "time_cycles": last_row["time_cycles"],
        }

        for col in var_cols:
            x = window_df[col].values.astype(np.float64)

            # ---- Baseline rolling stats ----
            x_mean = x.mean()
            x_std = x.std(ddof=0)
            x_min = x.min()
            x_max = x.max()
            x_skew = skew(x)
            x_kurt = kurtosis(x)

            row_dict[f"{col}_mean"] = x_mean
            row_dict[f"{col}_std"] = x_std
            row_dict[f"{col}_min"] = x_min
            row_dict[f"{col}_max"] = x_max
            row_dict[f"{col}_skew"] = x_skew
            row_dict[f"{col}_kurt"] = x_kurt

            # ---- fslope feature ----
            row_dict[f"{col}_fslope"] = compute_fslope(
                x_window=x,
                t_idx=t_idx,
                t_mean=t_mean,
                denom_t=denom_t,
                epsilon=epsilon,
            )

            # ---- LF-ER feature ----
            row_dict[f"{col}_lfer"] = compute_lfer(
                x_window=x,
                low_freq_ratio=low_freq_ratio,
                epsilon=epsilon,
            )

        # Label (RUL at last cycle in the window)
        row_dict["RUL"] = last_row["RUL"]

        rows.append(row_dict)

    features_df = pd.DataFrame(rows)
    return features_df


def generate_rolling_features_fslope_lfer(
    df,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,
    epsilon=EPSILON,
):
    """
    Wrapper over generate_rolling_features_for_unit_fslope_lfer for all engines.

    Parameters
    ----------
    df : pd.DataFrame
        Full labeled dataset (train_labeled or test_with_dummy_rul),
        containing 'unit_nr', 'time_cycles', 'RUL', SENSOR_COLS, SETTING_COLS.
    window : int
        Sliding window length.
    stride : int
        Sliding window step.
    low_freq_ratio : float
        Fraction of non-DC FFT bins treated as "low frequency".
    epsilon : float
        Small constant for numerical stability.

    Returns
    -------
    features_all : pd.DataFrame
        Concatenation of per-unit features, sorted by unit_nr and time_cycles.
    """
    all_features = []

    for unit_id, df_unit in df.groupby("unit_nr"):
        feats_unit = generate_rolling_features_for_unit_fslope_lfer(
            df_unit,
            window=window,
            stride=stride,
            low_freq_ratio=low_freq_ratio,
            epsilon=epsilon,
        )
        if not feats_unit.empty:
            all_features.append(feats_unit)

    if not all_features:
        return pd.DataFrame()

    features_all = pd.concat(all_features, axis=0, ignore_index=True)
    features_all = features_all.sort_values(["unit_nr", "time_cycles"]).reset_index(drop=True)
    return features_all


print("Combined fslope + LF-ER rolling feature generators are defined.")

Combined fslope + LF-ER rolling feature generators are defined.


In [21]:
# Cell 18 – Train feature extraction with combined fslope + LF-ER features

start_time = time.time()

# Generate sliding-window features with:
#   baseline stats + fslope + LF-ER
train_features_fslope_lfer = generate_rolling_features_fslope_lfer(
    train_labeled,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,  # same LF band definition as before
    epsilon=EPSILON,
)

end_time = time.time()
fslope_lfer_train_time = end_time - start_time

print("Proposed (fslope + LF-ER) TRAIN feature extraction completed.")
print(f"Proposed (fslope + LF-ER) TRAIN feature extraction time: {fslope_lfer_train_time:.3f} seconds")
print(f"train_features_fslope_lfer shape: {train_features_fslope_lfer.shape}")

# Identify feature columns (exclude ID/label columns)
non_feature_cols_fslope_lfer = ["unit_nr", "time_cycles", "RUL"]
feature_cols_fslope_lfer = [
    c for c in train_features_fslope_lfer.columns
    if c not in non_feature_cols_fslope_lfer
]

print(f"Number of features (fslope + LF-ER): {len(feature_cols_fslope_lfer)}")
print("Example feature columns (first 20):")
print(feature_cols_fslope_lfer[:20])

C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\1908113684.py:73: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_skew = skew(x)
C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\1908113684.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_kurt = kurtosis(x)


Proposed (fslope + LF-ER) TRAIN feature extraction completed.
Proposed (fslope + LF-ER) TRAIN feature extraction time: 809.753 seconds
train_features_fslope_lfer shape: (17731, 195)
Number of features (fslope + LF-ER): 192
Example feature columns (first 20):
['s_1_mean', 's_1_std', 's_1_min', 's_1_max', 's_1_skew', 's_1_kurt', 's_1_fslope', 's_1_lfer', 's_2_mean', 's_2_std', 's_2_min', 's_2_max', 's_2_skew', 's_2_kurt', 's_2_fslope', 's_2_lfer', 's_3_mean', 's_3_std', 's_3_min', 's_3_max']


In [22]:
# Cell 19 – Build X, y, groups for the combined fslope + LF-ER pipeline

# Non-feature columns: identifiers and label
non_feature_cols_fslope_lfer = ["unit_nr", "time_cycles", "RUL"]

# Rebuild feature_cols_fslope_lfer defensively (in case cells are re-run out of order)
feature_cols_fslope_lfer = [
    c for c in train_features_fslope_lfer.columns
    if c not in non_feature_cols_fslope_lfer
]

# Design matrix (features), target (RUL), and groups (unit_nr)
X_fslope_lfer = train_features_fslope_lfer[feature_cols_fslope_lfer].values
y_fslope_lfer = train_features_fslope_lfer["RUL"].values
groups_fslope_lfer = train_features_fslope_lfer["unit_nr"].values

print("Proposed (fslope + LF-ER) design matrix constructed.")
print(f"X_fslope_lfer shape      : {X_fslope_lfer.shape}")
print(f"y_fslope_lfer shape      : {y_fslope_lfer.shape}")
print(f"groups_fslope_lfer shape : {groups_fslope_lfer.shape}")
print(f"Number of features (fslope + LF-ER): {len(feature_cols_fslope_lfer)}")
print("Example feature columns (first 20):")
print(feature_cols_fslope_lfer[:20])

Proposed (fslope + LF-ER) design matrix constructed.
X_fslope_lfer shape      : (17731, 192)
y_fslope_lfer shape      : (17731,)
groups_fslope_lfer shape : (17731,)
Number of features (fslope + LF-ER): 192
Example feature columns (first 20):
['s_1_mean', 's_1_std', 's_1_min', 's_1_max', 's_1_skew', 's_1_kurt', 's_1_fslope', 's_1_lfer', 's_2_mean', 's_2_std', 's_2_min', 's_2_max', 's_2_skew', 's_2_kurt', 's_2_fslope', 's_2_lfer', 's_3_mean', 's_3_std', 's_3_min', 's_3_max']


In [23]:
# Cell 20 – GroupKFold CV with XGBoost (combined fslope + LF-ER features)

# Reuse rmse() from earlier; if not defined (e.g., cells run out of order), define it here.
try:
    rmse
except NameError:
    def rmse(y_true, y_pred):
        return np.sqrt(mean_squared_error(y_true, y_pred))


gkf_fslope_lfer = GroupKFold(n_splits=5)

rmse_scores_fslope_lfer = []
mae_scores_fslope_lfer = []

cv_start_time_fslope_lfer = time.time()

for fold, (train_idx, val_idx) in enumerate(
    gkf_fslope_lfer.split(X_fslope_lfer, y_fslope_lfer, groups_fslope_lfer),
    start=1
):
    X_tr, X_val = X_fslope_lfer[train_idx], X_fslope_lfer[val_idx]
    y_tr, y_val = y_fslope_lfer[train_idx], y_fslope_lfer[val_idx]

    model_fslope_lfer_cv = XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="reg:squarederror",
        n_jobs=-1,
        random_state=RANDOM_STATE,
    )

    model_fslope_lfer_cv.fit(X_tr, y_tr)
    y_val_pred = model_fslope_lfer_cv.predict(X_val)

    fold_rmse = rmse(y_val, y_val_pred)
    fold_mae = mean_absolute_error(y_val, y_val_pred)

    rmse_scores_fslope_lfer.append(fold_rmse)
    mae_scores_fslope_lfer.append(fold_mae)

    print(f"Fold {fold}: RMSE = {fold_rmse:.3f}, MAE = {fold_mae:.3f}")

cv_end_time_fslope_lfer = time.time()
cv_time_fslope_lfer = cv_end_time_fslope_lfer - cv_start_time_fslope_lfer

rmse_mean_fslope_lfer = np.mean(rmse_scores_fslope_lfer)
rmse_std_fslope_lfer = np.std(rmse_scores_fslope_lfer)
mae_mean_fslope_lfer = np.mean(mae_scores_fslope_lfer)
mae_std_fslope_lfer = np.std(mae_scores_fslope_lfer)

print("\nProposed (fslope + LF-ER) CV results:")
print(f"  RMSE mean = {rmse_mean_fslope_lfer:.3f}, std = {rmse_std_fslope_lfer:.3f}")
print(f"  MAE  mean = {mae_mean_fslope_lfer:.3f}, std = {mae_std_fslope_lfer:.3f}")
print(f"Proposed (fslope + LF-ER) CV time (5 folds): {cv_time_fslope_lfer:.3f} seconds")

Fold 1: RMSE = 38.486, MAE = 23.791
Fold 2: RMSE = 32.607, MAE = 20.742
Fold 3: RMSE = 31.632, MAE = 20.434
Fold 4: RMSE = 33.243, MAE = 22.333
Fold 5: RMSE = 29.480, MAE = 20.574

Proposed (fslope + LF-ER) CV results:
  RMSE mean = 33.090, std = 2.984
  MAE  mean = 21.575, std = 1.303
Proposed (fslope + LF-ER) CV time (5 folds): 20.373 seconds


In [24]:
# Cell 21 – Final XGBoost model trained on full fslope + LF-ER feature set

model_fslope_lfer = XGBRegressor(
    n_estimators=400,           # like other final models
    max_depth=6,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=RANDOM_STATE,
)

start_time = time.time()

# Train on ALL combined-feature windows
model_fslope_lfer.fit(X_fslope_lfer, y_fslope_lfer)

end_time = time.time()
fslope_lfer_train_model_time = end_time - start_time

print("Final proposed (fslope + LF-ER) model trained successfully.")
print(f"Training time (full fslope + LF-ER features): {fslope_lfer_train_model_time:.3f} seconds")
print(f"Number of training samples: {X_fslope_lfer.shape[0]}")
print(f"Number of features        : {X_fslope_lfer.shape[1]}")

Final proposed (fslope + LF-ER) model trained successfully.
Training time (full fslope + LF-ER features): 5.908 seconds
Number of training samples: 17731
Number of features        : 192


In [25]:
# Cell 22 – TEST feature extraction with combined fslope + LF-ER features

# Ensure we have a test DataFrame with a dummy RUL column
def add_dummy_rul(test_df):
    """
    Add a dummy RUL column to the test set so that the same
    rolling feature generator can be reused.

    The dummy RUL will later be replaced with the true RUL values
    from rul_test_final for the last cycle of each engine.
    """
    df = test_df.copy()
    df["RUL"] = 0  # placeholder, not used directly for test labeling
    return df


# If test_with_dummy_rul already exists (from fslope/LF-ER), reuse it; otherwise create it.
if "test_with_dummy_rul" not in globals():
    test_with_dummy_rul = add_dummy_rul(test_raw)

# Time the TEST feature extraction with combined fslope + LF-ER
start_time = time.time()

test_features_fslope_lfer_full = generate_rolling_features_fslope_lfer(
    test_with_dummy_rul,
    window=WINDOW_SIZE,
    stride=STRIDE,
    low_freq_ratio=0.25,
    epsilon=EPSILON,
)

end_time = time.time()
fslope_lfer_test_time = end_time - start_time

print("Proposed (fslope + LF-ER) TEST feature extraction completed.")
print(f"Proposed (fslope + LF-ER) TEST feature extraction time: {fslope_lfer_test_time:.3f} seconds")
print(f"test_features_fslope_lfer_full shape: {test_features_fslope_lfer_full.shape}")

C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\1908113684.py:73: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_skew = skew(x)
C:\Users\hrast\AppData\Local\Temp\ipykernel_26660\1908113684.py:74: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  x_kurt = kurtosis(x)


Proposed (fslope + LF-ER) TEST feature extraction completed.
Proposed (fslope + LF-ER) TEST feature extraction time: 451.989 seconds
test_features_fslope_lfer_full shape: (10196, 195)


In [26]:
# Cell 23 – TEST evaluation for the combined fslope + LF-ER model

# 1) For each test engine, keep ONLY the last available window
idx_last_per_unit_fslope_lfer = (
    test_features_fslope_lfer_full.groupby("unit_nr")["time_cycles"].idxmax()
)

test_features_fslope_lfer_last = (
    test_features_fslope_lfer_full
        .loc[idx_last_per_unit_fslope_lfer]
        .sort_values("unit_nr")
        .reset_index(drop=True)
)

# 2) Replace dummy RUL with true final RUL from rul_test_final
test_features_fslope_lfer_last["RUL"] = rul_test_final["RUL"].reset_index(drop=True)

# 3) Build X_test and y_test using the SAME feature columns as training
X_test_fslope_lfer = test_features_fslope_lfer_last[feature_cols_fslope_lfer].values
y_test_fslope_lfer = test_features_fslope_lfer_last["RUL"].values

# 4) Predict with the final combined model and compute metrics
y_test_fslope_lfer_pred = model_fslope_lfer.predict(X_test_fslope_lfer)

test_rmse_fslope_lfer = rmse(y_test_fslope_lfer, y_test_fslope_lfer_pred)
test_mae_fslope_lfer  = mean_absolute_error(y_test_fslope_lfer, y_test_fslope_lfer_pred)

print("Proposed (fslope + LF-ER) TEST evaluation:")
print(f"  test_features_fslope_lfer_last shape: {test_features_fslope_lfer_last.shape}")
print(f"  X_test_fslope_lfer shape            : {X_test_fslope_lfer.shape}")
print(f"  y_test_fslope_lfer shape            : {y_test_fslope_lfer.shape}")
print(f"  TEST RMSE (fslope + LF-ER)          : {test_rmse_fslope_lfer:.3f}")
print(f"  TEST MAE  (fslope + LF-ER)          : {test_mae_fslope_lfer:.3f}")

Proposed (fslope + LF-ER) TEST evaluation:
  test_features_fslope_lfer_last shape: (100, 195)
  X_test_fslope_lfer shape            : (100, 192)
  y_test_fslope_lfer shape            : (100,)
  TEST RMSE (fslope + LF-ER)          : 23.241
  TEST MAE  (fslope + LF-ER)          : 17.623


In [27]:
# Cell 24 – Parallel TRAIN feature extraction for fslope (HPC variant)

from joblib import Parallel, delayed

def generate_rolling_features_fslope_parallel(
    df,
    window=WINDOW_SIZE,
    stride=STRIDE,
    epsilon=EPSILON,
    n_jobs=-1,
):
    """
    Parallel version of generate_rolling_features_fslope for fslope-only pipeline.

    Strategy:
      - Split the full DataFrame by unit_nr.
      - For each unit, run generate_rolling_features_for_unit_fslope(...) in parallel.
      - Concatenate all per-unit feature DataFrames.
      - Sort by unit_nr and time_cycles for consistency with the serial version.

    Parameters
    ----------
    df : pd.DataFrame
        Full labeled dataset (e.g., train_labeled or test_with_dummy_rul),
        containing 'unit_nr', 'time_cycles', 'RUL', SENSOR_COLS, SETTING_COLS.
    window : int
        Sliding window length.
    stride : int
        Sliding window step.
    epsilon : float
        Small constant for fslope normalization.
    n_jobs : int
        Number of parallel workers (use -1 for all available cores).

    Returns
    -------
    features_all : pd.DataFrame
        Concatenation of per-unit features, sorted by unit_nr and time_cycles.
    """
    # Group once to avoid repeated grouping inside Parallel
    grouped_units = list(df.groupby("unit_nr"))

    # Run per-unit feature extraction in parallel
    results = Parallel(n_jobs=n_jobs, backend="loky")(
        delayed(generate_rolling_features_for_unit_fslope)(
            df_unit,
            window=window,
            stride=stride,
            epsilon=epsilon,
        )
        for unit_id, df_unit in grouped_units
    )

    # Filter out empty DataFrames (units with too few cycles)
    non_empty = [r for r in results if r is not None and not r.empty]
    if not non_empty:
        return pd.DataFrame()

    features_all = pd.concat(non_empty, axis=0, ignore_index=True)
    features_all = features_all.sort_values(["unit_nr", "time_cycles"]).reset_index(drop=True)
    return features_all


# -------------------------------------------------------------------
# Benchmark: serial vs parallel TRAIN fslope feature extraction
# -------------------------------------------------------------------

# If you already ran the serial fslope extraction earlier, we will reuse
# its timing; otherwise we recompute it here for a fair comparison.
if "train_features_fslope" not in globals():
    print("Serial fslope features not found in memory – recomputing for comparison...")
    start_serial = time.time()
    train_features_fslope = generate_rolling_features_fslope(
        train_labeled,
        window=WINDOW_SIZE,
        stride=STRIDE,
        epsilon=EPSILON,
    )
    end_serial = time.time()
    fslope_train_time = end_serial - start_serial
else:
    print("Using existing serial fslope features for timing comparison...")

print(f"Serial TRAIN (fslope) time: {fslope_train_time:.3f} seconds")
print(f"Serial TRAIN (fslope) shape: {train_features_fslope.shape}")

# Now run the parallel version
start_parallel = time.time()
train_features_fslope_parallel = generate_rolling_features_fslope_parallel(
    train_labeled,
    window=WINDOW_SIZE,
    stride=STRIDE,
    epsilon=EPSILON,
    n_jobs=-1,   # use all available cores
)
end_parallel = time.time()
fslope_train_time_parallel = end_parallel - start_parallel

print("\nParallel TRAIN (fslope) feature extraction completed.")
print(f"Parallel TRAIN (fslope) time: {fslope_train_time_parallel:.3f} seconds")
print(f"Parallel TRAIN (fslope) shape: {train_features_fslope_parallel.shape}")

# Optional sanity check: shapes and columns should match
if train_features_fslope.shape == train_features_fslope_parallel.shape:
    print("Shape check: OK (serial and parallel outputs have the same shape).")
else:
    print("WARNING: Shape mismatch between serial and parallel fslope features!")
    
if list(train_features_fslope.columns) == list(train_features_fslope_parallel.columns):
    print("Column check: OK (serial and parallel outputs have the same columns).")
else:
    print("WARNING: Column mismatch between serial and parallel fslope features!")

Using existing serial fslope features for timing comparison...
Serial TRAIN (fslope) time: 665.316 seconds
Serial TRAIN (fslope) shape: (17731, 171)

Parallel TRAIN (fslope) feature extraction completed.
Parallel TRAIN (fslope) time: 82.049 seconds
Parallel TRAIN (fslope) shape: (17731, 171)
Shape check: OK (serial and parallel outputs have the same shape).
Column check: OK (serial and parallel outputs have the same columns).


In [28]:
# Cell 25 – Parallel TEST feature extraction for fslope (HPC variant)

from joblib import Parallel, delayed  # safe even if already imported

# Reuse add_dummy_rul from earlier; redefine defensively if needed
try:
    add_dummy_rul
except NameError:
    def add_dummy_rul(test_df):
        df = test_df.copy()
        df["RUL"] = 0
        return df

# Ensure we have test_with_dummy_rul
if "test_with_dummy_rul" not in globals():
    test_with_dummy_rul = add_dummy_rul(test_raw)

# If we already have serial fslope TEST features/timing, reuse them; otherwise compute
if "test_features_fslope_full" not in globals() or "fslope_test_time" not in globals():
    print("Serial TEST (fslope) features not found in memory – recomputing for comparison...")
    start_serial_test = time.time()
    test_features_fslope_full = generate_rolling_features_fslope(
        test_with_dummy_rul,
        window=WINDOW_SIZE,
        stride=STRIDE,
        epsilon=EPSILON,
    )
    end_serial_test = time.time()
    fslope_test_time = end_serial_test - start_serial_test
else:
    print("Using existing serial TEST (fslope) features for timing comparison...")

print(f"Serial TEST (fslope) time : {fslope_test_time:.3f} seconds")
print(f"Serial TEST (fslope) shape: {test_features_fslope_full.shape}")

# Now run the parallel version on TEST
start_parallel_test = time.time()
test_features_fslope_full_parallel = generate_rolling_features_fslope_parallel(
    test_with_dummy_rul,
    window=WINDOW_SIZE,
    stride=STRIDE,
    epsilon=EPSILON,
    n_jobs=-1,   # all cores
)
end_parallel_test = time.time()
fslope_test_time_parallel = end_parallel_test - start_parallel_test

print("\nParallel TEST (fslope) feature extraction completed.")
print(f"Parallel TEST (fslope) time : {fslope_test_time_parallel:.3f} seconds")
print(f"Parallel TEST (fslope) shape: {test_features_fslope_full_parallel.shape}")

# Sanity checks: shapes & columns
if test_features_fslope_full.shape == test_features_fslope_full_parallel.shape:
    print("Shape check: OK (serial and parallel TEST outputs have the same shape).")
else:
    print("WARNING: Shape mismatch between serial and parallel TEST fslope features!")

if list(test_features_fslope_full.columns) == list(test_features_fslope_full_parallel.columns):
    print("Column check: OK (serial and parallel TEST outputs have the same columns).")
else:
    print("WARNING: Column mismatch between serial and parallel TEST fslope features!")

Using existing serial TEST (fslope) features for timing comparison...
Serial TEST (fslope) time : 383.331 seconds
Serial TEST (fslope) shape: (10196, 171)

Parallel TEST (fslope) feature extraction completed.
Parallel TEST (fslope) time : 48.699 seconds
Parallel TEST (fslope) shape: (10196, 171)
Shape check: OK (serial and parallel TEST outputs have the same shape).
Column check: OK (serial and parallel TEST outputs have the same columns).


In [29]:
# Cell 26 – Parquet caching for fslope features (TRAIN + TEST)

import os

# Paths where we'll store the cached features
FSLOPE_TRAIN_PARQUET = "features_train_fslope.parquet"
FSLOPE_TEST_PARQUET  = "features_test_fslope.parquet"

def save_fslope_features_to_parquet(
    train_df,
    test_df,
    train_path=FSLOPE_TRAIN_PARQUET,
    test_path=FSLOPE_TEST_PARQUET,
):
    """
    Save fslope feature tables (TRAIN and TEST) to Parquet files.

    Parameters
    ----------
    train_df : pd.DataFrame
        TRAIN fslope feature DataFrame.
    test_df : pd.DataFrame
        TEST fslope feature DataFrame.
    train_path : str
        File path for TRAIN Parquet.
    test_path : str
        File path for TEST Parquet.
    """
    train_df.to_parquet(train_path, index=False)
    test_df.to_parquet(test_path, index=False)
    print(f"Saved TRAIN fslope features to: {train_path}")
    print(f"Saved TEST  fslope features to: {test_path}")


def load_fslope_features_from_parquet(
    train_path=FSLOPE_TRAIN_PARQUET,
    test_path=FSLOPE_TEST_PARQUET,
):
    """
    Load fslope feature tables (TRAIN and TEST) from Parquet if they exist.

    Returns
    -------
    train_df, test_df : (pd.DataFrame, pd.DataFrame) or (None, None)
        Loaded DataFrames if files exist; otherwise (None, None).
    """
    if not (os.path.exists(train_path) and os.path.exists(test_path)):
        print("No cached Parquet features found.")
        return None, None

    train_df = pd.read_parquet(train_path)
    test_df = pd.read_parquet(test_path)
    print(f"Loaded TRAIN fslope features from: {train_path}")
    print(f"Loaded TEST  fslope features from: {test_path}")
    return train_df, test_df


# -------------------------------------------------------------------
# Example usage in this notebook
# -------------------------------------------------------------------

# Prefer the *parallel* versions if available, otherwise fall back
train_fslope_for_cache = (
    train_features_fslope_parallel if "train_features_fslope_parallel" in globals()
    else train_features_fslope
)
test_fslope_for_cache = (
    test_features_fslope_full_parallel if "test_features_fslope_full_parallel" in globals()
    else test_features_fslope_full
)

print("Caching current fslope TRAIN/TEST features to Parquet...")
save_fslope_features_to_parquet(
    train_fslope_for_cache,
    test_fslope_for_cache,
)

# To demonstrate loading (optional sanity check):
print("\nReloading fslope TRAIN/TEST features from Parquet...")
train_fslope_cached, test_fslope_cached = load_fslope_features_from_parquet()

if train_fslope_cached is not None and test_fslope_cached is not None:
    print(f"Cached TRAIN shape: {train_fslope_cached.shape}")
    print(f"Cached TEST  shape: {test_fslope_cached.shape}")

Caching current fslope TRAIN/TEST features to Parquet...
Saved TRAIN fslope features to: features_train_fslope.parquet
Saved TEST  fslope features to: features_test_fslope.parquet

Reloading fslope TRAIN/TEST features from Parquet...
Loaded TRAIN fslope features from: features_train_fslope.parquet
Loaded TEST  fslope features from: features_test_fslope.parquet
Cached TRAIN shape: (17731, 171)
Cached TEST  shape: (10196, 171)


In [30]:
# Cell 27 – Reload fslope features from Parquet and rebuild X, y, groups

# Make sure the helper and paths exist (from Cell 26); redefine defensively if needed
import os

try:
    FSLOPE_TRAIN_PARQUET
    FSLOPE_TEST_PARQUET
except NameError:
    FSLOPE_TRAIN_PARQUET = "features_train_fslope.parquet"
    FSLOPE_TEST_PARQUET  = "features_test_fslope.parquet"

def load_fslope_features_from_parquet(
    train_path=FSLOPE_TRAIN_PARQUET,
    test_path=FSLOPE_TEST_PARQUET,
):
    """
    Load fslope feature tables (TRAIN and TEST) from Parquet if they exist.

    Returns
    -------
    train_df, test_df : (pd.DataFrame, pd.DataFrame) or (None, None)
        Loaded DataFrames if files exist; otherwise (None, None).
    """
    if not (os.path.exists(train_path) and os.path.exists(test_path)):
        print("No cached Parquet features found.")
        return None, None

    train_df = pd.read_parquet(train_path)
    test_df = pd.read_parquet(test_path)
    print(f"Loaded TRAIN fslope features from: {train_path}")
    print(f"Loaded TEST  fslope features from: {test_path}")
    return train_df, test_df


# -------------------------------------------------------------------
# Reload cached fslope features
# -------------------------------------------------------------------
train_fslope_cached, test_fslope_cached = load_fslope_features_from_parquet()

if train_fslope_cached is None or test_fslope_cached is None:
    print("Cannot rebuild X/y/groups from cache – Parquet files are missing.")
else:
    # Rebuild feature columns (exclude ID/label columns)
    non_feature_cols_fslope = ["unit_nr", "time_cycles", "RUL"]
    feature_cols_fslope = [
        c for c in train_fslope_cached.columns
        if c not in non_feature_cols_fslope
    ]

    # TRAIN design matrix
    X_fslope = train_fslope_cached[feature_cols_fslope].values
    y_fslope = train_fslope_cached["RUL"].values
    groups_fslope = train_fslope_cached["unit_nr"].values

    # TEST design matrix
    X_test_fslope = test_fslope_cached[feature_cols_fslope].values
    # For test, the cached fslope features may still have dummy RUL;
    # if you want true RULs, you can overwrite from rul_test_final later.
    # Here we just pull what's in the cached file:
    y_test_fslope = test_fslope_cached["RUL"].values

    print("Rebuilt fslope design matrices from cached Parquet features.")
    print(f"TRAIN: X_fslope shape      : {X_fslope.shape}")
    print(f"TRAIN: y_fslope shape      : {y_fslope.shape}")
    print(f"TRAIN: groups_fslope shape : {groups_fslope.shape}")
    print(f"TEST : X_test_fslope shape : {X_test_fslope.shape}")
    print(f"TEST : y_test_fslope shape : {y_test_fslope.shape}")
    print(f"# features (fslope)        : {len(feature_cols_fslope)}")

Loaded TRAIN fslope features from: features_train_fslope.parquet
Loaded TEST  fslope features from: features_test_fslope.parquet
Rebuilt fslope design matrices from cached Parquet features.
TRAIN: X_fslope shape      : (17731, 168)
TRAIN: y_fslope shape      : (17731,)
TRAIN: groups_fslope shape : (17731,)
TEST : X_test_fslope shape : (10196, 168)
TEST : y_test_fslope shape : (10196,)
# features (fslope)        : 168


In [31]:
# Cell 28 – Summary table of all pipelines (Baseline vs Proposed)

# If numpy/pandas aren't in scope for some reason, re-import (safe)
import numpy as np
import pandas as pd

# -------------------------------------------------------------------
# 1) Baseline metrics (from your baseline notebook / earlier notes)
#    You can tweak these if you rerun the baseline.
# -------------------------------------------------------------------
baseline_cv_rmse_mean = 39.9
baseline_cv_rmse_std  = 3.1
baseline_cv_mae_mean  = 27.3
baseline_cv_mae_std   = 1.8

baseline_test_rmse = 29.009
baseline_test_mae  = 21.226

# If you know baseline feature extraction times from the baseline notebook,
# you can plug them in here; otherwise we keep them as NaN.
baseline_train_time_serial = np.nan   # e.g. replace with your baseline_train_time
baseline_test_time_serial  = np.nan   # e.g. replace with your baseline_test_time

# -------------------------------------------------------------------
# 2) Collect metrics for each pipeline
# -------------------------------------------------------------------
rows = []

# Baseline (stats only)
rows.append({
    "Pipeline": "Baseline (stats only)",
    "CV_RMSE_mean": baseline_cv_rmse_mean,
    "CV_RMSE_std":  baseline_cv_rmse_std,
    "CV_MAE_mean":  baseline_cv_mae_mean,
    "CV_MAE_std":   baseline_cv_mae_std,
    "Test_RMSE":    baseline_test_rmse,
    "Test_MAE":     baseline_test_mae,
    "Train_feat_time_serial_s": baseline_train_time_serial,
    "Test_feat_time_serial_s":  baseline_test_time_serial,
    "Train_feat_time_parallel_s": np.nan,
    "Test_feat_time_parallel_s":  np.nan,
})

# Proposed: Baseline + fslope
rows.append({
    "Pipeline": "Proposed: stats + fslope",
    "CV_RMSE_mean": rmse_mean_fslope,
    "CV_RMSE_std":  rmse_std_fslope,
    "CV_MAE_mean":  mae_mean_fslope,
    "CV_MAE_std":   mae_std_fslope,
    "Test_RMSE":    test_rmse_fslope,
    "Test_MAE":     test_mae_fslope,
    "Train_feat_time_serial_s":   fslope_train_time,
    "Test_feat_time_serial_s":    fslope_test_time,
    "Train_feat_time_parallel_s": fslope_train_time_parallel,
    "Test_feat_time_parallel_s":  fslope_test_time_parallel,
})

# Proposed: Baseline + LF-ER
rows.append({
    "Pipeline": "Proposed: stats + LF-ER",
    "CV_RMSE_mean": rmse_mean_lfer,
    "CV_RMSE_std":  rmse_std_lfer,
    "CV_MAE_mean":  mae_mean_lfer,
    "CV_MAE_std":   mae_std_lfer,
    "Test_RMSE":    test_rmse_lfer,
    "Test_MAE":     test_mae_lfer,
    "Train_feat_time_serial_s":   lfer_train_time,
    "Test_feat_time_serial_s":    lfer_test_time,
    "Train_feat_time_parallel_s": np.nan,
    "Test_feat_time_parallel_s":  np.nan,
})

# Proposed: Baseline + fslope + LF-ER
rows.append({
    "Pipeline": "Proposed: stats + fslope + LF-ER",
    "CV_RMSE_mean": rmse_mean_fslope_lfer,
    "CV_RMSE_std":  rmse_std_fslope_lfer,
    "CV_MAE_mean":  mae_mean_fslope_lfer,
    "CV_MAE_std":   mae_std_fslope_lfer,
    "Test_RMSE":    test_rmse_fslope_lfer,
    "Test_MAE":     test_mae_fslope_lfer,
    "Train_feat_time_serial_s":   fslope_lfer_train_time,
    "Test_feat_time_serial_s":    fslope_lfer_test_time,
    "Train_feat_time_parallel_s": np.nan,
    "Test_feat_time_parallel_s":  np.nan,
})

summary_df = pd.DataFrame(rows)

# Nice ordering of columns
summary_df = summary_df[
    [
        "Pipeline",
        "CV_RMSE_mean", "CV_RMSE_std",
        "CV_MAE_mean",  "CV_MAE_std",
        "Test_RMSE",    "Test_MAE",
        "Train_feat_time_serial_s", "Test_feat_time_serial_s",
        "Train_feat_time_parallel_s", "Test_feat_time_parallel_s",
    ]
]

print("Summary of Baseline and Proposed Pipelines:")
summary_df

Summary of Baseline and Proposed Pipelines:


,Pipeline,CV_RMSE_mean,CV_RMSE_std,CV_MAE_mean,CV_MAE_std,Test_RMSE,Test_MAE,Train_feat_time_serial_s,Test_feat_time_serial_s,Train_feat_time_parallel_s,Test_feat_time_parallel_s
0,Baseline (stats only),39.900000,3.100000,27.300000,1.800000,29.009000,21.226000,NaN,NaN,NaN,NaN
1,Proposed: stats + fslope,33.301382,2.732285,21.602949,1.369023,22.349753,17.190367,665.316446,383.331138,82.048785,48.699486
2,Proposed: stats + LF-ER,37.637002,3.169968,25.186698,1.612716,27.919778,20.035334,691.600941,395.505026,NaN,NaN
3,Proposed: stats + fslope + LF-ER,33.089605,2.984159,21.574733,1.302642,23.241497,17.622917,809.752789,451.988945,NaN,NaN
